In [3]:
import pandas as pd
import numpy as np

In [4]:
d = pd.read_csv('../tables/2024_03_11_merged_03.csv', parse_dates=['Time_Mid'])

In [5]:
bin_labels = ['SMPS_Bin01', 'SMPS_Bin02', 'SMPS_Bin03', 'SMPS_Bin04', 'SMPS_Bin05',
       'SMPS_Bin06', 'SMPS_Bin07', 'SMPS_Bin08', 'SMPS_Bin09', 'SMPS_Bin10',
       'SMPS_Bin11', 'SMPS_Bin12', 'SMPS_Bin13', 'SMPS_Bin14', 'SMPS_Bin15',
       'SMPS_Bin16', 'SMPS_Bin17', 'SMPS_Bin18', 'SMPS_Bin19', 'SMPS_Bin20',
       'SMPS_Bin21', 'SMPS_Bin22', 'SMPS_Bin23', 'SMPS_Bin24', 'SMPS_Bin25',
       'SMPS_Bin26', 'SMPS_Bin27', 'SMPS_Bin28', 'SMPS_Bin29', 'SMPS_Bin30', 'LAS_Bin01', 'LAS_Bin02', 'LAS_Bin03', 'LAS_Bin04', 'LAS_Bin05',
       'LAS_Bin06', 'LAS_Bin07', 'LAS_Bin08', 'LAS_Bin09', 'LAS_Bin10',
       'LAS_Bin11', 'LAS_Bin12', 'LAS_Bin13', 'LAS_Bin14', 'LAS_Bin15',
       'LAS_Bin16', 'LAS_Bin17', 'LAS_Bin18', 'LAS_Bin19', 'LAS_Bin20',
       'LAS_Bin21', 'LAS_Bin22', 'LAS_Bin23', 'LAS_Bin24', 'LAS_Bin25',
       'LAS_Bin26']

lower_bounds = [2.97, 3.36, 3.76, 4.22, 4.74, 5.32, 5.97, 6.69, 7.51, 8.43, 9.46, 10.6, 11.9, 13.4, 15.0, 16.8, 18.9, 21.2, 23.8, 26.7, 29.9, 33.6, 37.6, 42.2, 47.4, 53.2, 59.7, 66.9, 75.1, 84.3, 93.9, 106.1, 119.0, 133.6, 149.9, 168.2, 188.7, 211.7, 237.5, 266.5, 299.0, 335.5, 376.5, 422.4, 473.9, 531.8, 596.6, 669.5, 751.1, 842.8, 945.6, 1129.5, 1421.9, 1790.1, 2253.6, 2837.1]
upper_bounds = [3.36, 3.76, 4.22, 4.74, 5.32, 5.97, 6.69, 7.51, 8.43, 9.46, 10.6, 11.9, 13.4, 15.0, 16.8, 18.9, 21.2, 23.8, 26.7, 29.9, 33.6, 37.6, 42.2, 47.4, 53.2, 59.7, 66.9, 75.1, 84.3, 94.0, 106.1, 119.0, 133.6, 149.9, 168.2, 188.7, 211.7, 237.5, 266.5, 299.0, 335.5, 376.5, 422.4, 473.9, 531.8, 596.6, 669.5, 751.1, 842.8, 945.6, 1129.5, 1421.9, 1790.1, 2253.6, 2837.1, 3487.5]

dlogd=[]
for i in range(0,len(lower_bounds)):
       dlogd.append(np.log10(upper_bounds[i]) - np.log10(lower_bounds[i]))

d['N_tot'] = (d[bin_labels] * dlogd).sum(axis=1)
d['OMF'] = d['Org_Ave_IsoK_STP'] / d['ams_tot']
d['f_SMPS_LAS'] = d[bin_labels].notna().mean(axis=1)

In [6]:
d = d[d['f_SMPS_LAS'] > .95]

In [7]:
d = d.dropna(subset=['N_CCN_stdPT', 'N_tot', 'D_a', 'k_obs', 'OMF'])

In [8]:
# Get the count of each unique value in the 'CCN_SS' column
value_counts = d['CCN_SS'].value_counts()

# Convert the Series to a DataFrame for better presentation
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['Unique Value', 'Count']
value_counts_df.sort_values(by='Unique Value', inplace=True)
value_counts_df

,Unique Value,Count
66,0.160,9
92,0.165,6
23,0.170,21
90,0.175,6
40,0.180,14
...,...,...
101,0.695,5
48,0.700,12
98,0.705,6
22,0.710,22


In [9]:
d = d[(d['CCN_SS'] >= 0.37) & (d['CCN_SS'] <= 0.43)]

In [10]:
transit_flights = [pd.Timestamp('2022-03-22').date(), pd.Timestamp('2022-05-18').date(), pd.Timestamp('2022-05-21').date(), pd.Timestamp('2022-05-31').date(), pd.Timestamp('2022-06-18').date()]
df_winter = d[d['Time_Mid'].dt.month.isin([11, 12, 1, 2, 3, 4]) & (~d['Time_Mid'].dt.date.isin(transit_flights))]
df_spring = d[d['Time_Mid'].dt.month.isin([5, 6]) & (np.logical_not((d['Time_Mid'].dt.month.isin([6])) & (d['Time_Mid'].dt.year.isin([2022])))) & (~d['Time_Mid'].dt.date.isin(transit_flights))]
df_summer = d[d['Time_Mid'].dt.month.isin([8, 9]) & (~d['Time_Mid'].dt.date.isin(transit_flights))]
df_bermuda = d[(d['Time_Mid'].dt.month.isin([6])) & (d['Time_Mid'].dt.year.isin([2022])) & (~d['Time_Mid'].dt.date.isin(transit_flights))]

In [11]:
ds = [df_winter, df_spring, df_summer, df_bermuda]

In [12]:
d_out = pd.DataFrame(columns = ['N', 'N_CCN', 'N_tot', 'D_c', 'k_obs', 'OMF'])

In [13]:
def print_cell(d, dgts=0):
    return f"{d.median():.{dgts}f} ({d.quantile(.25):.{dgts}f}, {d.quantile(.75):.{dgts}f})"

In [14]:
for i in range(0,4):
    d_out.loc[i,'N'] = ds[i].shape[0]
    d_out.loc[i,'N_CCN'] = print_cell(ds[i]['N_CCN_stdPT'])
    d_out.loc[i,'N_tot'] = print_cell(ds[i]['N_tot'])
    d_out.loc[i,'D_c'] = print_cell(ds[i]['D_a'], 1)
    d_out.loc[i,'k_obs'] = print_cell(ds[i]['k_obs'], 2)
    d_out.loc[i,'OMF'] = print_cell(ds[i]['OMF'], 2)

In [15]:
d_out

,N,N_CCN,N_tot,D_c,k_obs,OMF
0,4969,"460 (224, 817)","1596 (827, 3294)","75.0 (64.6, 88.7)","0.29 (0.17, 0.47)","0.46 (0.35, 0.59)"
1,3407,"488 (272, 907)","1037 (556, 2142)","78.0 (65.0, 86.4)","0.27 (0.20, 0.47)","0.58 (0.33, 0.73)"
2,33,"652 (239, 1344)","1576 (968, 2367)","74.7 (64.1, 105.4)","0.29 (0.10, 0.49)","0.64 (0.59, 0.74)"
3,785,"309 (246, 421)","486 (382, 639)","46.0 (41.2, 52.1)","1.33 (0.91, 1.85)","0.26 (0.17, 0.44)"


In [16]:
d_out.drop(index=2).to_excel('../tables/summary_stats.xlsx', index=False)